This notebook calculates the topic model of the craving set, it clusters into 10 different topics

In [24]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
import nltk
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [25]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

Change the folder either to `instagram_data/` or to `twitter_data/`

In [26]:
folder  = 'instagram_data/' #twitter_data/ or instagram_data/

In [27]:
data = pd.read_csv(folder +'posts_processed.csv')
green = pd.read_csv(folder +'green_posts_processed.csv')

/var/folders/pv/j2wxhdx50y5fbwqrc2yxqwf00000gn/T/ipykernel_67301/1928277876.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(folder +'posts_processed.csv')


In [28]:
df = green #either data or green
df

,platformId,platform,date,updated,type,post_text,postUrl,subscriberCount,score,media,...,username,actual,likes,comments,Branche,Branche-6,Sektor,B2B,B2C,green
0,355091021065555690_10145706,Instagram,2012-12-26 19:29:29,2022-06-23 22:21:38,photo,Pro Green X #PUMA tee @professorgreen #PUMAlife,https://www.instagram.com/p/TtiWb9iw7q/,0,-57.086404,"[{'type': 'photo', 'url': 'https://scontent-yy...",...,puma,"{'favoriteCount': 775, 'commentCount': 12}",775,12,mode,Textil,2.0,True,True,True
1,346237364735773953_10145706,Instagram,2012-12-14 14:18:51,2022-06-23 22:21:38,photo,A day in the life of #PUMA #pumalife - morning...,https://www.instagram.com/p/TOFQx9Cw0B/,0,-29.654785,"[{'type': 'photo', 'url': 'https://scontent-yy...",...,puma,"{'favoriteCount': 1486, 'commentCount': 29}",1486,29,mode,Textil,2.0,True,True,True
2,342685722404917046_10145706,Instagram,2012-12-09 16:42:23,2022-06-23 22:21:38,photo,"#PUMA, always looking good our friend @profess...",https://www.instagram.com/p/TBdtlRiw82/,0,-79.799290,"[{'type': 'photo', 'url': 'https://scontent-yy...",...,puma,"{'favoriteCount': 557, 'commentCount': 6}",557,6,mode,Textil,2.0,True,True,True
3,333882232069275743_25749975,Instagram,2012-11-27 13:11:25,2022-05-27 23:50:24,photo,Mercedes-Benz SLS AMG Coupé Black Series! #mer...,https://www.instagram.com/p/SiMB7_TcBf/,0,-27.810777,"[{'type': 'photo', 'url': 'https://scontent-se...",...,mercedesbenz,"{'favoriteCount': 3219, 'commentCount': 10}",3219,10,automobil,Automobil,2.0,True,True,True
4,329423030961684553_25749975,Instagram,2012-11-21 09:31:47,2022-05-27 23:50:24,photo,Say hello to the newest @mercedesamg powerhous...,https://www.instagram.com/p/SSWIBVTcBJ/,0,-22.427822,"[{'type': 'photo', 'url': 'https://scontent-se...",...,mercedesbenz,"{'favoriteCount': 3979, 'commentCount': 25}",3979,25,automobil,Automobil,2.0,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9333,2742155642375737703_43109246,Instagram,2022-01-02 08:00:47,2022-07-07 23:43:38,album,A powerful start to the day.\nThe BMW M5 CS.\n...,https://www.instagram.com/p/CYOGAc2o5Fn/,32377665,2.185437,"[{'type': 'photo', 'url': 'https://scontent-se...",...,bmw,"{'favoriteCount': 232218, 'commentCount': 695}",232218,695,automobil,Automobil,2.0,True,True,True
9334,2741880971927861036_25749975,Instagram,2022-01-01 22:55:30,2022-05-27 23:40:36,photo,Say hello to renewable adventures.\n\n📸 @lotte...,https://www.instagram.com/p/CYNHjeEKbcs/,34682189,-1.155339,"[{'type': 'photo', 'url': 'https://scontent-se...",...,mercedesbenz,"{'favoriteCount': 77451, 'commentCount': 276}",77451,276,automobil,Automobil,2.0,True,True,True
9335,2741715737230173911_462752227,Instagram,2022-01-01 17:26:46,2022-11-30 12:30:21,album,Commander of the fjords: the Macan gets to kno...,https://www.instagram.com/p/CYMh-_Qs-bX/,26065549,2.848996,"[{'type': 'photo', 'url': 'https://scontent-se...",...,porsche,"{'favoriteCount': 239375, 'commentCount': 462}",239375,462,automobil,Automobil,2.0,True,True,True
9336,2741566902972241322_43109246,Instagram,2022-01-01 12:31:04,2022-07-07 23:43:39,photo,Make this your first decision of 2022 🤔 \nThe ...,https://www.instagram.com/p/CYMAJKjoNWq/,32367896,1.504407,"[{'type': 'photo', 'url': 'https://scontent-se...",...,bmw,"{'favoriteCount': 158901, 'commentCount': 521}",158901,521,automobil,Automobil,2.0,True,True,True


In [29]:
data = df[df['post_text'].str.len()>4]
# Convert to list
data = data.post_text.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', post) for post in data]

# Remove distracting single quotes
data = [re.sub("\'", "", post) for post in data]

pprint(data[:1])

['Pro Green X #PUMA tee @professorgreen #PUMAlife']


In [30]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

data_words = [[word for word in words if len(word)>2] for words in data_words]    

print(data_words[:1])

[['pro', 'green', 'puma', 'tee', 'professorgreen', 'pumalife']]


In [31]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['pro', 'green', 'puma', 'tee', 'professorgreen', 'pumalife']


In [33]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [34]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# if it does not work, run "python3 -m spacy download en" in command line
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pro', 'green', 'puma', 'tee', 'professorgreen', 'pumalife']]


In [35]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [36]:
id2word[0]

'green'

In [37]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('green', 1),
  ('pro', 1),
  ('professorgreen', 1),
  ('puma', 1),
  ('pumalife', 1),
  ('tee', 1)]]

In [38]:
import time
# Build LDA model
numTopicsRange = [3,4,5,7,10,12,15]
validationPerplexity = []
coherences = []
umass_coherences = []
uci_coherences = []
npmi_coherences = []
timeElapsed = []
lda_models = []
for i in numTopicsRange:
    start = time.time()
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    end = time.time()
    validationPerplexity.append(lda_model.log_perplexity(corpus))
    coherences.append(CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v').get_coherence())
    umass_coherences.append(CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='u_mass').get_coherence())
    uci_coherences.append(CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_uci').get_coherence())
    npmi_coherences.append(CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_npmi').get_coherence())
    lda_models.append(lda_model)
    timeElapsed.append(end-start) 

In [39]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(figsize=(9, 3.5), dpi=1600)
seconds = [t/100 for t in timeElapsed]
plt.plot(numTopicsRange,validationPerplexity, color = 'r', label = 'Validation perplexity')
#plt.plot(numTopicsRange,seconds, color = 'b', label = 'time elapsed')
plt.xticks(numTopicsRange)
plt.xlabel("Number of Topics")
plt.legend(loc='upper left')
plt.show()

In [40]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(figsize=(9, 3.5), dpi=1600)
seconds = [t/100 for t in timeElapsed]
plt.plot(numTopicsRange,coherences, color = 'b', label = 'Coherence Score')
#plt.plot(numTopicsRange,seconds, color = 'b', label = 'time elapsed')
plt.xticks(numTopicsRange)
plt.xlabel("Number of Topics")
plt.legend(loc='upper left')
plt.show()

In [41]:
# Print the Keyword in the 10 topics
best_idx = 4
pprint(lda_models[best_idx].print_topics())
doc_lda = lda_models[best_idx][corpus]

[(0,
  '0.126*"find" + 0.051*"charge" + 0.048*"available" + 0.042*"passenger" + '
  '0.026*"official" + 0.021*"specific" + 0.015*"racing" + 0.014*"favourite" + '
  '0.014*"deutsche" + 0.013*"information"'),
 (1,
  '0.051*"accord" + 0.049*"regulation" + 0.047*"figure" + 0.039*"vehicle" + '
  '0.033*"version" + 0.030*"applicable" + 0.028*"determined" + 0.027*"car" + '
  '0.023*"interior" + 0.020*"equivalent"'),
 (2,
  '0.094*"performance" + 0.059*"high" + 0.040*"race" + 0.035*"season" + '
  '0.021*"anniversary" + 0.017*"roll" + 0.017*"high_quality" + 0.015*"limited" '
  '+ 0.015*"face" + 0.012*"combination"'),
 (3,
  '0.409*"datum" + 0.099*"bmwelectric_electricvehicle" + '
  '0.092*"electriccar_zeroemission" + 0.050*"business" + 0.018*"call" + '
  '0.009*"pmcgphoto" + 0.009*"extra" + 0.008*"apply" + 0.006*"motion" + '
  '0.005*"thei_bornelectric"'),
 (4,
  '0.202*"combine" + 0.133*"consumption" + 0.110*"emission" + 0.106*"fuel" + '
  '0.068*"com_disclaimer" + 0.059*"accord" + 0.035*"leit

In [42]:
print(lda_models[best_idx].top_topics(corpus))

[([(0.050884504, 'accord'), (0.04878576, 'regulation'), (0.046611153, 'figure'), (0.038581036, 'vehicle'), (0.03284744, 'version'), (0.029912604, 'applicable'), (0.027848866, 'determined'), (0.026809605, 'car'), (0.023037946, 'interior'), (0.019781642, 'equivalent'), (0.01947257, 'history'), (0.018917356, 'respect'), (0.015811618, 'duty'), (0.01482755, 'approval'), (0.014782337, 'may_differ'), (0.0147532, 'comparison'), (0.014751168, 'directive'), (0.014736767, 'least_inter'), (0.014719741, 'classification'), (0.01471005, 'order_ensure')], -1.1453925200409998), ([(0.033074867, 'info'), (0.022884177, 'click'), (0.0225365, 'sustainable'), (0.013913301, 'year'), (0.013815736, 'use'), (0.012782163, 'link'), (0.011163869, 'make'), (0.011108777, 'world'), (0.009801879, 'work'), (0.009514646, 'help'), (0.009337139, 'future'), (0.008993124, 'hydrogen'), (0.0074137533, 'innovation'), (0.0071201795, 'also'), (0.0064211492, 'today'), (0.006388115, 'story'), (0.0062697656, 'need'), (0.0061565223, 

In [43]:
# Compute Perplexity
print('\nPerplexity: ', lda_models[best_idx].log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_models[best_idx], texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.2820392534449025

Coherence Score:  0.45102165230320657


In [44]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_models[best_idx], corpus, id2word)
vis

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.301896 -0.161671       1        1  22.552046
9      0.102890  0.348161       2        1  18.049098
5      0.209853 -0.155824       3        1  17.231370
7      0.123981  0.193254       4        1  12.021928
1     -0.098212 -0.066898       5        1  10.393989
0     -0.144323 -0.029074       6        1   9.488411
8     -0.050047 -0.079385       7        1   5.998549
6     -0.131811 -0.024988       8        1   2.142157
2     -0.161224 -0.010450       9        1   1.446626
3     -0.153004 -0.013125      10        1   0.675824, topic_info=                 Term          Freq         Total Category  logprob  loglift
22            combine  14287.000000  14287.000000  Default  30.0000  30.0000
23        consumption   9417.000000   9417.000000  Default  29.0000  29.0000
25           emission   7769.000000   7769.000000  Default  28.0000  28.0000
26               fuel   7508.000000   7508.000000  Default  27.0000  27.0000
472              find   3755.000000   3755.000000  Default  26.0000  26.0000
...               ...           ...           ...      ...      ...      ...
14257  saopauloyellow      0.072636      2.720885  Topic10 -10.2796   1.3737
1386            crisp      0.059766      2.336798  Topic10 -10.4747   1.3309
13228         loalesi      0.078485      3.262909  Topic10 -10.2022   1.2695
13812     active_tour      0.224229     59.700673  Topic10  -9.1524  -0.5874
13753     bmwelectric      0.117657     10.205879  Topic10  -9.7973   0.5340

[361 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
2908       1  0.716271       accord
2908       5  0.283428       accord
2908       6  0.000171       accord
13812      1  0.954763  active_tour
13812      5  0.016750  active_tour
...      ...       ...          ...
238        5  0.001587        world
238        6  0.001587        world
366        2  0.996877         year
366        5  0.001268         year
366        6  0.001268         year

[918 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 6, 8, 2, 1, 9, 7, 3, 4])

In [45]:
# 1. Wordcloud of Top N words in each topic
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=3500,
                  height=1800,
                  max_words=14,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_models[best_idx].show_topics(formatted=False)

fig, axes = plt.subplots(2, 5, figsize=(50,50), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=500)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.figure(figsize=(9, 3.5), dpi=1600)
plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()
'''plt.savefig("wordclouds", 
                          dpi=3600,
                          facecolor='w',
                          edgecolor='w',
                          orientation='portrait',
                          papertype=None, 
                          format=None,
                          transparent=False,
                          bbox_inches=None, 
                          pad_inches=0.1,
                          frameon=None, 
                          metadata=None)'''

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
pyLDAvis.save_html(vis, 'lda.html')